In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
! head work/sample.csv

In [ ]:
df = spark.read.format("csv").option("header","true").option("delimiter", "|").load("work/sample.csv")
df.printSchema()
df.show(truncate=False)

In [ ]:
from pyspark.sql.functions import split, col


df = df.withColumn("city", split(col("city"), ","))
df.printSchema()
df.show(truncate=False)

In [ ]:
from pyspark.sql.functions import explode


df_exploded = df.withColumn("city", explode(col("city")))
df_exploded.printSchema()
df_exploded.show(truncate=False)

In [ ]:
from pyspark.sql.functions import collect_set


df_imploded = df_exploded.groupBy("name").agg(collect_set(col("city")))
df_imploded.printSchema()
df_imploded.show(truncate=False)

# Another example

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, col

# Initialize a Spark session
spark = SparkSession.builder.appName("explodeExample").getOrCreate()

# Sample data
data = [
    ("PMI", "OPO", [2, 1]),
    ("ATH", "BCN", [3]),
    ("JFK", "MAD", [5, 4, 6]),
    ("HND", "LAX", [8, 9, 7, 0])
]

# Create DataFrame
df = spark.createDataFrame(data, ["origin", "destination", "internal_flight_ids"])


In [ ]:
df.printSchema()

In [ ]:
# Transform the DataFrame
df_exploded = df.withColumn("internal_flight_ids", explode(col("internal_flight_ids")))

# Show the result
df_exploded.show()